In [2]:
import multiprocessing
import numpy as np
from sklearn.metrics import roc_curve, auc 
from sklearn.metrics import confusion_matrix
import seaborn as sb
import matplotlib.pyplot as plt
import pandas as pd
import sklearn as sk
import numpy as np

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import mean_squared_error
from sklearn import metrics
from sklearn.model_selection import cross_validate
from sklearn.utils import resample
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import RandomOverSampler
oversample = RandomOverSampler(sampling_strategy = 0.5)

with multiprocessing.Pool(processes = 6) as pool:
    result = pool.map(np.square, [1,2,3,4,5])

print(result)

[1, 4, 9, 16, 25]


In [9]:
#### Response of sensitivity and specificity to threshold


CIP_data = pd.read_csv("CIP_data_encode_prev.csv")
#bootstrap data
no_iterations = 1
bootstrapped_stats = []

#1. Create model using all data and get ROC_AUC ("ROC_AUC_neural_network")

model = MLPClassifier(solver = 'lbfgs', activation = 'tanh', max_iter = 2000 ,hidden_layer_sizes= 12, random_state=10, learning_rate = 'adaptive' )
model = MLPClassifier(solver = 'lbfgs', activation = 'tanh', max_iter = 300 ,hidden_layer_sizes= 12, alpha = 1.291549665014884, random_state=10, learning_rate = 'adaptive' )


##threshold
threshold_seq = [0,0.05,0.1,0.15,0.2,0.25,0.3,0.35,0.4,0.45,0.5,0.55,0.6,0.65,0.7,0.75,0.8,0.85,0.9,0.95,1]
threshold_seq = [0,0.5,1]

#functionalise
def bootstrapping(CIP_data, threshold_seq, model, no_iteratons):
    sensitivity_sample_table = []
    specificity_sample_table = []
    sensitivity_optimised_table = []
    specificity_optimised_table = []
    sensitivity_optimised_table_975 = []
    sensitivity_optimised_table_025 = []
    specificity_optimised_table_975 = []
    specificity_optimised_table_025 = []

    for threshold in threshold_seq:
        bootstrapped_stats = []
        #1. Create model using all data and get ROC_AUC ("ROC_AUC_neural_network") - need to do this for each threshold

        X = CIP_data[['MSMW', 'MSW', 'Oth/Unk/Missing','Northeast', 'Southeast', 'Southwest', 'West', 'PREV_REGION', 'PREV_CLINIC']]
        y = CIP_data['Susceptible']
        X, y = oversample.fit_resample(X,y)
        model_fit = model.fit(X, y)
        y_predict = model_fit.predict(X)
        y_predict_proba = model_fit.predict_proba(X)
        
        y_predict = np.where(y_predict_proba[:, 1] > threshold, 1, 0)
        tn_apparent , fp_apparent, fn_apparent, tp_apparent = confusion_matrix(y_true=y, y_pred=y_predict).ravel()

        sensitivity_apparent = tp_apparent / (tp_apparent  + fn_apparent )
        specificity_apparent  = tn_apparent / (tn_apparent + fp_apparent )
        for i in range(no_iteratons):
                #2. (A) Sample all individuals w/replacement
                sample = CIP_data.sample(frac = 1, replace=True) ##(a) sample n individuals with replacement
                X_sample = sample[['MSMW', 'MSW', 'Oth/Unk/Missing','Northeast', 'Southeast', 'Southwest', 'West', 'PREV_REGION', 'PREV_CLINIC']]
                y_sample = sample['Susceptible']
                X_sample, y_sample = oversample.fit_resample(X_sample,y_sample)


                #  (B) Develop predictive model and find apparent performance
                model_fit = model.fit(X_sample, y_sample)
                y_sample_predict = model_fit.predict(X_sample)
                y_sample_predict_prob = model_fit.predict_proba(X_sample)

                ROC_AUC_neural_network_bootstrap_sample_performance = metrics.roc_auc_score(y_sample, y_sample_predict) 

                #  (C) Performance of predictive model on original sample (i.e. original population, X)
                y_test_predict = model_fit.predict(X)
                y_test_predict_prob = model_fit.predict_proba(X)

                ROC_AUC_neural_network_bootstrap_test_performance = metrics.roc_auc_score(y, y_test_predict) ## 0.756384214489288
                ### (D) Calculate optimisation by getting (B) - (D) 
                optimism = ROC_AUC_neural_network_bootstrap_sample_performance - ROC_AUC_neural_network_bootstrap_test_performance
            
                ### (i) Calculate sensitivity and specificity 
                
                y_sample_predict = np.where(y_sample_predict_prob[:, 1] > threshold, 1, 0)
                y_test_predict = np.where(y_test_predict_prob[:, 1] > threshold, 1, 0)

                tn_test, fp_test, fn_test, tp_test = confusion_matrix(y_true=y, y_pred=y_test_predict).ravel()
                tn_sample, fp_sample, fn_sample, tp_sample = confusion_matrix(y_true=y_sample, y_pred=y_sample_predict).ravel()

                sensitivity_test = tp_test / (tp_test + fn_test)
                specificity_test = tn_test / (tn_test + fp_test)

                sensitivity_sample = tp_sample/ (tp_sample + fn_sample)
                specificity_sample = tn_sample / (tn_sample + fp_sample)

                fpr_test, tpr_test, threshold_test = roc_curve(y, y_test_predict_prob[:, 1], drop_intermediate=False)
                fpr_sample, tpr_sample, threshold_sample = roc_curve(y_sample, y_sample_predict_prob[:, 1], drop_intermediate=False)
                
                sensitivity_optimism = sensitivity_sample - sensitivity_test
                specificity_optimism = specificity_sample - specificity_test

                bootstrapped_stats.append(
                {
                    'Sample ROC': ROC_AUC_neural_network_bootstrap_sample_performance,
                    'Test ROC': ROC_AUC_neural_network_bootstrap_test_performance,
                    'Optimisation': optimism,
                    'Sample Sensitivity': sensitivity_sample,
                    'Sample Specificity': specificity_sample,
                    'Test Sensitivity': sensitivity_test,
                    'Test Specificity': specificity_test,
                    'Sensitivity Optimisation': sensitivity_optimism,
                    'Specificity Optimisation': specificity_optimism,
                    'Sample FPR': fpr_sample,
                    'Sample TPR': tpr_sample,
                    'Test FPR': fpr_test,
                    'Test TPR': tpr_test
                })
        bootstrapped_stats = pd.DataFrame(bootstrapped_stats)
        specificity_sample_optimism = bootstrapped_stats["Specificity Optimisation"].mean() 
        sensitivity_sample_optimism = bootstrapped_stats["Sensitivity Optimisation"].mean() 

        specificity_sample_optimised = specificity_apparent - specificity_sample_optimism ##
        sensitivity_sample_optimised = sensitivity_apparent - sensitivity_sample_optimism ##

        specificity_sample_95CI_lower = specificity_sample_optimised - bootstrapped_stats["Specificity Optimisation"].quantile(0.025)
        specificity_sample_95CI_upper = specificity_sample_optimised + bootstrapped_stats["Specificity Optimisation"].quantile(0.975)

        sensitivity_sample_95CI_lower= sensitivity_sample_optimised - bootstrapped_stats["Sensitivity Optimisation"].quantile(0.025)
        sensitivity_sample_95CI_upper = sensitivity_sample_optimised + bootstrapped_stats["Sensitivity Optimisation"].quantile(0.975)

    #sensitivity_sample_table.append(sensitivity_sample_optimised)
    #specificity_sample_table.append(specificity_sample_optimised)
    sensitivity_optimised_table.append(sensitivity_sample_optimised)
    specificity_optimised_table.append(specificity_sample_optimised)
    sensitivity_optimised_table_975.append(sensitivity_sample_95CI_upper)
    sensitivity_optimised_table_025.append(sensitivity_sample_95CI_lower)
    specificity_optimised_table_975.append(specificity_sample_95CI_upper)
    specificity_optimised_table_025.append(specificity_sample_95CI_lower)
    specificity_sensitivity_dataframe = pd.DataFrame({sensitivity_optimised_table, specificity_optimised_table,sensitivity_optimised_table_975, sensitivity_optimised_table_025, specificity_optimised_table_975,specificity_optimised_table_025})
    return specificity_sensitivity_dataframe

no_iteratons = 1
with multiprocessing.Pool(processes = 1) as pool:
    result = pool.map(bootstrapping(CIP_data, threshold_seq, model,no_iteratons))

/Users/rem76/miniconda3/envs/GISP_init/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:536: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
/Users/rem76/miniconda3/envs/GISP_init/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:536: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
/Users/rem76/miniconda3/envs/GISP_init/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:536: Conv

TypeError: unhashable type: 'list'

In [20]:

sensitivity_optimised_table = []
specificity_optimised_table = []
sensitivity_optimised_table_975 = []
sensitivity_optimised_table_025 = []
specificity_optimised_table_975 = []
specificity_optimised_table_025 = []
threshold_seq = [0,0.5,1]

for threshold in threshold_seq:
        bootstrapped_stats = []
        #1. Create model using all data and get ROC_AUC ("ROC_AUC_neural_network") - need to do this for each threshold

        X = CIP_data[['MSMW', 'MSW', 'Oth/Unk/Missing','Northeast', 'Southeast', 'Southwest', 'West', 'PREV_REGION', 'PREV_CLINIC']]
        y = CIP_data['Susceptible']
        X, y = oversample.fit_resample(X,y)
        model_fit = model.fit(X, y)
        y_predict = model_fit.predict(X)
        y_predict_proba = model_fit.predict_proba(X)
        
        y_predict = np.where(y_predict_proba[:, 1] > threshold, 1, 0)
        tn_apparent , fp_apparent, fn_apparent, tp_apparent = confusion_matrix(y_true=y, y_pred=y_predict).ravel()

        sensitivity_apparent = tp_apparent / (tp_apparent  + fn_apparent )
        specificity_apparent  = tn_apparent / (tn_apparent + fp_apparent )
        for i in range(no_iteratons):
                #2. (A) Sample all individuals w/replacement
                sample = CIP_data.sample(frac = 1, replace=True) ##(a) sample n individuals with replacement
                X_sample = sample[['MSMW', 'MSW', 'Oth/Unk/Missing','Northeast', 'Southeast', 'Southwest', 'West', 'PREV_REGION', 'PREV_CLINIC']]
                y_sample = sample['Susceptible']
                X_sample, y_sample = oversample.fit_resample(X_sample,y_sample)


                #  (B) Develop predictive model and find apparent performance
                model_fit = model.fit(X_sample, y_sample)
                y_sample_predict = model_fit.predict(X_sample)
                y_sample_predict_prob = model_fit.predict_proba(X_sample)

                ROC_AUC_neural_network_bootstrap_sample_performance = metrics.roc_auc_score(y_sample, y_sample_predict) 

                #  (C) Performance of predictive model on original sample (i.e. original population, X)
                y_test_predict = model_fit.predict(X)
                y_test_predict_prob = model_fit.predict_proba(X)

                ROC_AUC_neural_network_bootstrap_test_performance = metrics.roc_auc_score(y, y_test_predict) ## 0.756384214489288
                ### (D) Calculate optimisation by getting (B) - (D) 
                optimism = ROC_AUC_neural_network_bootstrap_sample_performance - ROC_AUC_neural_network_bootstrap_test_performance
            
                ### (i) Calculate sensitivity and specificity 
                
                y_sample_predict = np.where(y_sample_predict_prob[:, 1] > threshold, 1, 0)
                y_test_predict = np.where(y_test_predict_prob[:, 1] > threshold, 1, 0)

                tn_test, fp_test, fn_test, tp_test = confusion_matrix(y_true=y, y_pred=y_test_predict).ravel()
                tn_sample, fp_sample, fn_sample, tp_sample = confusion_matrix(y_true=y_sample, y_pred=y_sample_predict).ravel()

                sensitivity_test = tp_test / (tp_test + fn_test)
                specificity_test = tn_test / (tn_test + fp_test)

                sensitivity_sample = tp_sample/ (tp_sample + fn_sample)
                specificity_sample = tn_sample / (tn_sample + fp_sample)

                fpr_test, tpr_test, threshold_test = roc_curve(y, y_test_predict_prob[:, 1], drop_intermediate=False)
                fpr_sample, tpr_sample, threshold_sample = roc_curve(y_sample, y_sample_predict_prob[:, 1], drop_intermediate=False)
                
                sensitivity_optimism = sensitivity_sample - sensitivity_test
                specificity_optimism = specificity_sample - specificity_test

                bootstrapped_stats.append(
                {
                    'Sample ROC': ROC_AUC_neural_network_bootstrap_sample_performance,
                    'Test ROC': ROC_AUC_neural_network_bootstrap_test_performance,
                    'Optimisation': optimism,
                    'Sample Sensitivity': sensitivity_sample,
                    'Sample Specificity': specificity_sample,
                    'Test Sensitivity': sensitivity_test,
                    'Test Specificity': specificity_test,
                    'Sensitivity Optimisation': sensitivity_optimism,
                    'Specificity Optimisation': specificity_optimism,
                    'Sample FPR': fpr_sample,
                    'Sample TPR': tpr_sample,
                    'Test FPR': fpr_test,
                    'Test TPR': tpr_test
                })
        bootstrapped_stats = pd.DataFrame(bootstrapped_stats)
        specificity_sample_optimism = bootstrapped_stats["Specificity Optimisation"].mean() 
        sensitivity_sample_optimism = bootstrapped_stats["Sensitivity Optimisation"].mean() 

        specificity_sample_optimised = specificity_apparent - specificity_sample_optimism ##
        sensitivity_sample_optimised = sensitivity_apparent - sensitivity_sample_optimism ##

        specificity_sample_95CI_lower = specificity_sample_optimised - bootstrapped_stats["Specificity Optimisation"].quantile(0.025)
        specificity_sample_95CI_upper = specificity_sample_optimised + bootstrapped_stats["Specificity Optimisation"].quantile(0.975)

        sensitivity_sample_95CI_lower= sensitivity_sample_optimised - bootstrapped_stats["Sensitivity Optimisation"].quantile(0.025)
        sensitivity_sample_95CI_upper = sensitivity_sample_optimised + bootstrapped_stats["Sensitivity Optimisation"].quantile(0.975)

    #sensitivity_sample_table.append(sensitivity_sample_optimised)
    #specificity_sample_table.append(specificity_sample_optimised)
        sensitivity_optimised_table.append(sensitivity_sample_optimised)
        specificity_optimised_table.append(specificity_sample_optimised)
        sensitivity_optimised_table_975.append(sensitivity_sample_95CI_upper)
        sensitivity_optimised_table_025.append(sensitivity_sample_95CI_lower)
        specificity_optimised_table_975.append(specificity_sample_95CI_upper)
        specificity_optimised_table_025.append(specificity_sample_95CI_lower)

specificity_sensitivity_dataframe = pd.DataFrame({'sensitivity_test':sensitivity_optimised_table, 
'specificity_test': specificity_optimised_table,
'sensitivity_optimised_975':sensitivity_optimised_table_975, 
'sensitivity_optimised_025':sensitivity_optimised_table_025,
'sensitivity_optimised_975': specificity_optimised_table_975,
'specificity_optimised_025': specificity_optimised_table_025})

/Users/rem76/miniconda3/envs/GISP_init/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:536: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
/Users/rem76/miniconda3/envs/GISP_init/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:536: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
/Users/rem76/miniconda3/envs/GISP_init/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:536: Conv

TypeError: unhashable type: 'list'

In [17]:
print(type(sensitivity_optimised_table))

specificity_sensitivity_dataframe = pd.DataFrame({'sensitivity_test':sensitivity_optimised_table, 
'specificity_test': specificity_optimised_table,
'sensitivity_optimised_975':sensitivity_optimised_table_975, 
'sensitivity_optimised_025':sensitivity_optimised_table_025,
'sensitivity_optimised_975': specificity_optimised_table_975,
'specificity_optimised_025': specificity_optimised_table_025})

print(specificity_sensitivity_dataframe.head(6))


<class 'list'>
   sensitivity_test  specificity_test  sensitivity_test_975  \
0               0.0               1.0                   1.0   

   sensitivity_test_025  specificity_test_025  
0                   0.0                   1.0  
